# PGF for LDA

In [1]:
import algopy
from algopy import UTPM

import numpy as np
import numpy.random as rn

import pyaudi
from pyaudi import gdual_double as gdual

from scipy.stats import binom, multinomial, nbinom, poisson, gamma
from scipy.misc import factorial

### Define dummy data

In [2]:
# phi = np.array([[0.1, 0.8, 0.1], [0.5, 0.2, 0.3]]) # distribution over words
# theta = np.array([0.3, 0.7])                       # distribution over topics
# K, V = phi.shape      # K = number of topics, V = size of vocab
# N = 4                 # number of tokens in a document

N = 4   # number of tokens in a document
K = 2   # number of topics
V = 3   # number of unique word types

concen1 = 0.1 # concentration param. for topics.  when 0 < alpha < 1
              # topics are low entropy (i.e., peaked around a single val)
              # when alpha > 1, topics are high entropy
phi = rn.dirichlet(np.ones(V) * concen1, size=K)
assert phi.shape == (K, V)

concen2 = 1.  # concentration param. for document dist over topics
theta = rn.dirichlet(np.ones(K) * concen2)
assert theta.shape == (K,)

### True single-word marginals
Let $Y_v$ be the count of word type $w_v$ in a document, where $v$ is the word index. Compute the true $P(Y_v = y_v)$ for $y_v = 0, 1, .., N$.

In [3]:
p = theta.dot(phi).reshape(V, 1)
x = [range(N + 1) for _ in range(V)]
true_marginals = binom.pmf(x, N, p) # shape = (V, N + 1)

### PGF single-word marginals

In [4]:
def pgf_marginal(v, y_v, phi, theta): # Compute P(Y_v = y_v) = P(count(w_v) = y_v)
    D = y_v + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, 0] = 1
        
    u = algopy.ones(V, dtype=u_v)
    u[v] = u_v
    
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    return h.data[:, 0][y_v]

# Oberve 3 tokens of word type w_0
y_v, v = 3, 0
print 'PGF marginal:', pgf_marginal(v, y_v, phi, theta)
print 'True marginal:', true_marginals[0, 3]

PGF marginal: 0.013610074011
True marginal: 0.013610074011


Observe a document of length $N$ with word counts $y = [y_0, ..., y_V]$, where $\sum_{i=0}^V y_i = N$. Find the **single-word** marginal probabilities.

In [5]:
def pgf_marginals(y, phi, theta): # Compute [P(Y_0 = y[0]), ..., P(Y_V = y[V])]
    D = np.max(y) + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, :] = 1
    
    u = algopy.ones((V, V), dtype=u_v)
    np.fill_diagonal(u, u_v)
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    return [h_v.data[:, 0][y[i]] for i, h_v in enumerate(h)]

# Observe 2 tokens of w_0, 1 token of w_1, and 1 token of w_2
y = np.array([2, 1, 1])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.10768625211072375, 0.033325403389952588, 0.20171080757610418]
True marginals: [ 0.10768625  0.0333254   0.20171081]


In [6]:
# Observe 1 w_0, 0 w_1, and 3 w_2's
y = np.array([1, 0, 3])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.37868449125066855, 0.0023533241785814248, 0.00084784982471401509]
True marginals: [ 0.37868449  0.00235332  0.00084785]


In [7]:
# Observe 0 w_0, 4 w_1's, and 0 w_2
y = np.array([0, 4, 0])
print 'PGF marginals:', pgf_marginals(y, phi, theta)
print 'True marginals:', true_marginals[np.arange(V), y]

PGF marginals: [0.49937413470731667, 0.36967188698036285, 0.77781138106727588]
True marginals: [ 0.49937413  0.36967189  0.77781138]


### PGF joint marginals

In [8]:
def pgf_joint_marginal(v, w, y_v, y_w, phi, theta): # Compute P(Y_v = y_v, Y_w = y_w)
    # Init gdual objects
    order = y_v + y_w
    u_v = gdual(0, "v", order)
    u_w = gdual(0, "w", order)
    
    K, V = phi.shape
    u = [1] * V
    u[v] = u_v
    u[w] = u_w
    
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    
    # Evaluate the derivative
    return h.get_derivative([y_v, y_w])/(factorial(y_v) * factorial(y_w))

def true_joint_marginal(v, w, y_v, y_w, N, p):
    K, V = phi.shape
    y = np.array([y_v, y_w, N-(y_v+y_w)]) if N-(y_v+y_w) > 0 else np.array([y_v, y_w])
    p = np.array([p[v], p[w], 1-(p[v]+p[w])]) if N-(y_v+y_w) > 0 else np.array([p[v], p[w]])
    return multinomial.pmf(y, n=N, p=p)

y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal:', pgf_joint_marginal(v, w, y_v, y_w, phi, theta)
print 'True marginal:', true_joint_marginal(v, w, y_v, y_w, N, p.reshape(-1))

PGF marginal: 0.00552790104739
True marginal: 0.00552790104739


Integrate out N, $N \sim Poisson(\lambda)$.

In [9]:
def pgf_joint_marginal(v, w, y_v, y_w, phi, theta, lmbda): # Compute P(Y_v = y_v, Y_w = y_w)
    # Init gdual objects
    order = y_v + y_w
    u_v = gdual(0, "v", order)
    u_w = gdual(0, "w", order)
    
    K, V = phi.shape
    u = [1] * V
    u[v] = u_v
    u[w] = u_w
    
    t = phi.dot(u)
    s = theta.dot(t)
    h = pyaudi.exp(lmbda * (s - 1))
    
    # Evaluate the derivative
    return h.get_derivative([y_v, y_w])/(factorial(y_v) * factorial(y_w))

y_v, v = 4, 0 # 1 count of word 0
y_w, w = 0, 2 # 2 counts of word 2
lmbda = 3
print 'PGF marginal:', pgf_joint_marginal(v, w, y_v, y_w, phi, theta, lmbda)
print 'True marginal:', np.prod(poisson.pmf([y_v, y_w], lmbda*np.dot(theta, phi)[[0, 2]]))

PGF marginal: 0.00112435301704
True marginal: 0.00112435301704


Alogpy version.

In [10]:
def pgf_joint_marginal_algopy(v, w, y_v, y_w, phi, theta):
    """
    D = max order of differentiation + 1
    P = # of directions
    N = # of variables
    """
    D = y_v + y_w + 1
    P = 4 # (V + D - 2) choose D   
    
    # Init UTP objects
    u = UTPM(np.zeros((D,P,V)))
    u.data[0,:,1] = [1] * 4
    u.data[1,0] = [3, 0, 0]
    u.data[1,1] = [2, 0, 1]
    u.data[1,2] = [1, 0, 2]
    u.data[1,3] = [0, 0, 3]
    
    # Compute functions as usual
    t = phi.dot(u)
    s = theta.dot(t)
    h = np.power(s, N)
    
    # Use polarization identity to recover the mixed partial derivative
    c = h.data[-1]
    res = ((2*c[0]/9 - c[1] + 2*c[2] - 5*c[3]/9) / 3) / (factorial(y_v) * factorial(y_w))
    
    return res

# Test
y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal:', pgf_joint_marginal_algopy(v, w, y_v, y_w, phi, theta)
print 'True marginal:', true_joint_marginal(v, w, y_v, y_w, N, p.reshape(-1))

PGF marginal: 0.00552790104739
True marginal: 0.00552790104739


## With growth
Add growth:
- $n = $ number of tokens in the document (observed)
- $m = \sum_{i=1}^n x_i$, where $x_i \sim log(\rho)$
- $\mathbf{y} \sim mult(m, \boldsymbol{\theta}^T \mathbf{\Phi})$

In [11]:
rho = 0.3 # growth parameter
def pgf_marginal_growth(v, y_v, phi, theta, rho): # Compute P(Y_v = y_v) = P(count(w_v) = y_v)
    D = y_v + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, 0] = 1
        
    u = algopy.ones(V, dtype=u_v)
    u[v] = u_v
    t = phi.dot(u)
    s = theta.dot(t)
    r = np.log(1 - rho*s) / np.log(1 - rho)
    h = np.power(r, N)
    return h.data[:, 0][y_v]

# Oberve 3 tokens of word type w_0
y_v, v = 3, 0
print 'PGF marginal:', pgf_marginal_growth(v, y_v, phi, theta, rho)
#print 'True marginal:', true_marginals[0, 3]

PGF marginal: 0.027480036634


In [12]:
def pgf_marginals_growth(y, phi, theta, rho): # Compute [P(Y_0 = y[0]), ..., P(Y_V = y[V])]
    D = np.max(y) + 1
    u_v = UTPM(np.zeros((D, 1)))
    if D > 1:
        u_v.data[1, :] = 1
    
    u = algopy.ones((V, V), dtype=u_v)
    np.fill_diagonal(u, u_v)
    t = phi.dot(u)
    s = theta.dot(t)
    r = np.log(1 - rho*s) / np.log(1 - rho)
    h = np.power(r, N)
    return [h_v.data[:, 0][y[i]] for i, h_v in enumerate(h)]

# Observe 3 tokens of w_0, 1 token of w_1, and 1 token of w_2
y = np.array([3, 1, 1])
print 'PGF marginals:', pgf_marginals_growth(y, phi, theta, rho)

PGF marginals: [0.027480036634008973, 0.019775940563368573, 0.22775702779673998]


## NB LDA

In [13]:
a = 1    # shape parameter of theta_k
b = 0.5  # rate parameter of theta_k
phi_special = np.full((K, V), 1.0/V) # special case where distribution over word types is uniform

### Single-word marginals

In [14]:
def log_pgf(s, p):
    tmp = np.array([pyaudi.log(1 - p*s_k) for s_k in list(s)]) # b/c log(1-p*s) doesn't work
    return tmp / np.log(1 - p)

def pgf_marginal_nb(v, y_v, phi, a, b): # Compute P(Y_v = y_v)
    order = y_v
    K, V = phi.shape
    
    # Init gdual object
    u = [1] * V
    u_v = gdual(0, "v", order)
    u[v] = u_v
    
    # Compute the joint PGF
    t = phi.dot(u)
    s = log_pgf(t, 1.0 / (1+b))
    h = pyaudi.exp(a * (np.sum(s) - K) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    return h.get_derivative([y_v])/factorial(y_v)

def true_marginal_nb(v, y_v, phi, a, b): # only works for K = 2
    phi_v = phi[:, v].reshape((-1, 1))
    p1, p2 = nbinom.pmf([range(y_v + 1)] * 2, a, 1 - (phi_v/(b+phi_v)))
    return np.convolve(p1, p2)[y_v]

y_v, v = 5, 1

# Special case
print 'PGF marginal:', pgf_marginal_nb(v, y_v, phi_special, a, b)
print 'True marginal:', nbinom.pmf(y_v, K*a, 1 - (1.0 / (V*b + 1)))

# General case
print 'PGF marginal:', pgf_marginal_nb(v, y_v, phi, a, b)
print 'True marginal:', true_marginal_nb(v, y_v, phi, a, b)

PGF marginal: 0.0221184
True marginal: 0.0221184
PGF marginal: 0.0438946818231
True marginal: 0.0438946818231


### Joint marginals

In [15]:
def pgf_joint_marginal_nb(v, w, y_v, y_w, phi, a, b): # Compute P((Y_1, Y_2, Y_3) = y), assume V = 3
    order = y_v + y_w
    K, V = phi.shape
    
    # Init gdual objects
    u = [1] * V
    u[v] = gdual(0, "v", order)
    u[w] = gdual(0, "w", order)
    
    # Compute the joint PGF
    t = phi.dot(u)
    s = log_pgf(t, 1.0 / (1+b))
    h = pyaudi.exp(a * (np.sum(s) - K) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    y = [y_v, y_w]
    return h.get_derivative(y)/np.prod(factorial(y))

def sampled_joint_marginal_nb(v, w, y_v, y_w, phi, a, b, n_samples = 1000000):
    K, V = phi.shape
    S = n_samples # number of samples
    theta_hat = gamma.rvs(a, scale=1.0/b, size=(S, K))
    
    lmbda_hat = theta_hat.dot(phi)[:, [v, w]] # (S, 2)
    probs = poisson.pmf(np.array([y_v, y_w]).reshape((1, 2)), lmbda_hat)
    joint_probs = probs.prod(axis=1)
    return joint_probs.mean()

y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal:', pgf_joint_marginal_nb(v, w, y_v, y_w, phi_special, a, b)
print 'Sampled marginal:', sampled_joint_marginal_nb(v, w, y_v, y_w, phi_special, a, b)
print 'True marginal:', np.prod(nbinom.pmf(y, K*a, 1 - (1.0 / (V*b + 1))))

PGF marginal: 0.051407151782
Sampled marginal: 0.0514123330424
True marginal: 0.00764411904


Use algopy.

In [16]:
from itertools import product
from scipy.misc import comb

def compute_gamma(i, j):
    """
    i = order of differentiation for each variable (e.g. [1, 2])
    j = direction vector
    """
    i, j = np.array(i), np.array(j)
    
    if np.count_nonzero(j) > np.count_nonzero(i): return 0
    
    d = np.sum(j) # max total degree of differentiation of the UTP
    ranges = [range(i0+1) for i0 in i]
    ks = [k for k in product(*ranges)][1:]
        
    running_sum = 0
    for k in ks:
        k = np.array(k)
        sign = (-1)**np.sum(i - k)
        
        f1 = np.prod(comb(i, k))
        f2 = np.prod(comb(d*k/np.sum(k), j))
        print comb(d*k/np.sum(k), j)
        f3 = (float(np.sum(k))/d)**np.sum(i)
        
        term = sign*f1*f2*f3
        print i, j, k, term
        running_sum += term
        
    return running_sum

def get_derivative_algopy(x, i, dirs):
    """
    x    = a UTP object
    i    = order of differentiation for each variable (e.g. [1, 2])
    dirs = matrix of directions (in the same ordering as the directions of x)
    """
    
    order = np.sum(i) # total degree of differentiation
    c = x.data[order] # dth Taylor coefficients from all directions
    true_gammas = [2./27, -1./3, 2./3, -5./27]
    
    running_sum = 0
    for j, c0, true_gamma in zip(dirs, c, true_gammas):
        # j is a direction vector
        
        gamma = compute_gamma(i, j)
        term = gamma*c0
        print 'j, computed gamma_j, true gamma_j:', j, gamma, true_gamma
        running_sum += term
    
    return running_sum

def log_pgf_algopy(s, p):
    return np.log(1-p*s) / np.log(1 - p)

def pgf_joint_marginal_nb_algopy(v, w, y_v, y_w, phi, thetaa, b):
    """
    D = max order of differentiation + 1
    P = # of directions
    N = # of variables
    """
    K, V = phi.shape
    D = y_v + y_w + 1
    P = 4 # (V + D - 2) choose (D - 1)
    
    # Init UTP objects
    u = UTPM(np.zeros((D,P,V)))
    u.data[0,:,1] = [1] * 4
    dirs = np.array([[3, 0, 0],
                     [2, 0, 1],
                     [1, 0, 2],
                     [0, 0, 3]])
    u.data[1] = dirs
    
    # Compute functions as usual
    t = phi.dot(u)
    s = log_pgf_algopy(t, 1.0 / (1+b))
    lmbda = a * np.log(1 + (1.0/b))
    h = np.exp(lmbda * (np.sum(s) - K))
    
    # Use polarization identity to recover the mixed partial derivative
    c = h.data[-1]
    
    res = ((2*c[0]/9 - c[1] + 2*c[2] - 5*c[3]/9) / 3) 
    #res1 = get_derivative_algopy(h, [y_v, y_w], dirs[:, [0, 2]])
    
    #print res, res1
    
    return res / (factorial(y_v) * factorial(y_w))

# Test
y_v, v = 1, 0 # 1 count of word 0
y_w, w = 2, 2 # 2 counts of word 2
print 'PGF marginal pyaudi:', pgf_joint_marginal_nb(v, w, y_v, y_w, phi_special, a, b)
print 'PGF marginal algopy:', pgf_joint_marginal_nb_algopy(v, w, y_v, y_w, phi_special, a, b)
print 'Sampled marginal:', sampled_joint_marginal_nb(v, w, y_v, y_w, phi_special, a, b)
print 'True marginal:', np.prod(nbinom.pmf([y_v, y_w], K*a, 1 - (1.0 / (V*b + 1))))

PGF marginal pyaudi: 0.051407151782
PGF marginal algopy: 0.051407151782
Sampled marginal: 0.0513870331519
True marginal: 0.0497664


### Deep NB LDA

For d = 1 to D:
- $l_k^{(0)} \sim Poisson(a \ln(1 + 1/b))$
- $y_k^{(d)}|l_k^{(d-1)} \sim SumLog(l_k^{(d-1)}, 1/(1 + b))$ (I think this is wrong for $d > 1$)
- $\{y_{kv}^{(d)}\}_v \sim Mult(y_k^{(d)}, \{\phi_{kv}^{(d)}\}_v)$
- $l_k^{(d)} = \sum_k y_{kv}^{(d)}$
- $y_v = l_k^{(D)}$

In [17]:
# D = 4
K0, K1, K2, K3, V = 4, 4, 5, 5, 3
phi1 = rn.dirichlet(np.ones(K1) * 0.8, size=K0) # (K0, K1)
phi2 = rn.dirichlet(np.ones(K2) * 0.5, size=K1) # (K1, K2)
phi3 = rn.dirichlet(np.ones(K3) * 0.2, size=K2) # (K2, K3)
phi4 = rn.dirichlet(np.ones(V) * 0.1, size=K3) #(K3, V)

phi_layer = [phi1, phi2, phi3, phi4]

In [18]:
def pgf_marginal_deep(v, y_v, phi, a, b):
    order = y_v
    D = len(phi)
    K0, _ = phi[0].shape
    _, V = phi[-1].shape
    
    # Init gdual objects
    s = [1] * V
    s[v] = gdual(0, "v", order)
    
    # Compute the joint PGF
    for d in range(D, 0, -1):
        t = phi[d-1].dot(s)
        s = log_pgf(t, 1.0/ (1+b))
    h = pyaudi.exp(a * (np.sum(s) - K0) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    return h.get_derivative([y_v])/factorial(y_v)

v, y_v = 1, 60
print 'PGF marginal:', pgf_marginal_deep(v, y_v, phi_layer, a, b)

PGF marginal: 0.00497846202142


In [19]:
def pgf_joint_marginal_deep(y, phi, a, b):
    order = np.sum(y)
    D = len(phi)
    K0, _ = phi[0].shape
    _, V = phi[-1].shape
    
    # Init gdual objects
    s = [gdual(0, "v", order), gdual(0, "w", order), gdual(0, 'x', order)]
    
    # Compute the joint PGF
    for d in range(D, 0, -1):
        t = phi[d-1].dot(s)
        s = log_pgf(t, 1.0/ (1+b))
    h = pyaudi.exp(a * (np.sum(s) - K0) * np.log(1 + (1.0/b)))
    
    # Evaluate the derivative
    return h.get_derivative(y)/np.prod(factorial(y))

y = np.array([5, 20, 10])
print 'PGF marginal:', pgf_joint_marginal_deep(y, phi_layer, a, b)

PGF marginal: 5.06906763491e-05
